In [1]:
%load_ext autoreload
%autoreload

from lib.dataset import download_tfds_imdb_as_text, download_tfds_imdb_as_text_tiny
from lib.classical_ml import run_pipeline



In [2]:
dataset  = download_tfds_imdb_as_text()
tiny_dataset = download_tfds_imdb_as_text_tiny()

# Experiment 1 Tokenizer

In [3]:


def exp1(dataset):
    
    print("Simple SpaCy tokenizer")
    _, _ = run_pipeline(dataset)

    print("Simple SpaCy tokenizer and ignore stop")
    _, _ = run_pipeline(dataset, lower=True, ignore=["like_num", "is_stop"])
    
    print("Simple SpaCy tokenizer, lowercase, lemma")
    _, _ = run_pipeline(dataset, lower=True, lemma=True)
    
    

# approximate running time: 16 mins
import time
now = time.time()
exp1(dataset)
print(time.time()-now)

Simple SpaCy tokenizer
Load tokenized document from disk
Load tokenized document from disk


NameError: name 'is_logistic' is not defined

# Experiment 2 Vectorizer

Prerequisite: If you are not familiar with TFIDF, read see [this](https://nlp.stanford.edu/IR-book/pdf/06vect.pdf).

In this experiement, we will try different vectorization techniques; Bigrams, TFIDF and Binary. Although all of them are based one hot encoding, they capture slightly different information from text. 

- Bigrams:

- TFIDF: TFIDF is very common technique for Information Retrieval (IR) and has been long proved that it improves the IR performance. However, text classification and IR are two different problem sets, so this is not neccessary the case. For text classification, we can expect that whatever classification models we use can capture the same things as TFIDF, which is how importance words are, or how much information words contain.

- Binary:


In [ ]:


def exp2(dataset):
    
    print("Simple SpaCy tokenizer")
    _, _ = run_pipeline(dataset, lower=True, lemma=True, bigram=True)

    print("Simple SpaCy tokenizer and ignore stop")
    _, _ = run_pipeline(dataset, lower=True, lemma=True, tfidf=True)
      
    print("Simple SpaCy tokenizer, lowercase, lemma")
    _, _ = run_pipeline(dataset, lower=True, lemma=True, binary=True)
    
    

# approximate running time: 16 mins
import time
now = time.time()
exp2(dataset)
print(time.time()-now)

# Experiment 3 Model

- Naive Bayes
- Logisitc